# QuTech IVVI Manual Modules

This notebook demonstrates the usage of QCoDeS instrument drivers for manual QuTech IVVI rack modules. These drivers serve as software abstraction layers for documentation, parameter tracking, and integration with measurement scripts for modules that are operated manually.

## Overview

The IVVI manual modules complement the existing RS232-based IVVI driver by providing software representations of manual modules. They are designed for:

- **Documentation**: Keep track of module configurations and settings
- **Parameter tracking**: Monitor and validate parameter ranges  
- **Integration**: Seamless integration with QCoDeS measurement scripts
- **Extensibility**: Easy to add new module types

## Installation and Imports

First, let's import the necessary modules:

In [ ]:
import numpy as np
import time

from qcodes_contrib_drivers.drivers.QuTech.IVVI_Modules import (
    IVVI_Module, S4c, M2m, M2b, M1b, VId, IVd
)

## Available Modules

### Base Class: `IVVI_Module`

All manual modules inherit from this base class which provides:
- Common parameters: `module_type`, `rack_position`, `notes`
- Standard `get_idn()` method
- Consistent interface across all modules

In [ ]:
# Create a base module instance to demonstrate common functionality
base_module = IVVI_Module('demo_module')
base_module.module_type('Demo')
base_module.rack_position('Slot 1')
base_module.notes('Example module for demonstration')

print(f"Module name: {base_module.name}")
print(f"Module type: {base_module.module_type()}")
print(f"Rack position: {base_module.rack_position()}")
print(f"IDN: {base_module.get_idn()}")

## Current Source Modules

### `S4c` - 4-Channel Current Source
- **Range**: ±10 μA with 16-bit resolution
- **Channels**: 4 independent current outputs
- **Parameters**: 
  - `ch1_current` through `ch4_current` (range: ±10 μA)
  - `ch1_enabled` through `ch4_enabled` (boolean)
  - `current_range`, `resolution`

In [ ]:
# Create and configure a current source
current_source = S4c('cs1')
current_source.rack_position('Slot 1')
current_source.notes('Gate voltage control')

print(f"Current source: {current_source.name}")
print(f"Module type: {current_source.module_type()}")
print(f"Current range: {current_source.current_range()}")
print(f"Resolution: {current_source.resolution():.2e} A")

# Configure channels
current_source.ch1_enabled(True)
current_source.ch1_current(1e-6)  # 1 μA
current_source.ch2_enabled(True)
current_source.ch2_current(-0.5e-6)  # -0.5 μA

print(f"\nChannel 1: {current_source.ch1_current()} A, enabled: {current_source.ch1_enabled()}")
print(f"Channel 2: {current_source.ch2_current()} A, enabled: {current_source.ch2_enabled()}")

#### Parameter Validation

The modules automatically validate parameter ranges:

In [ ]:
# Demonstrate parameter validation
try:
    current_source.ch1_current(15e-6)  # Should fail: exceeds ±10 μA range
    print("This should not print - validation failed")
except ValueError as e:
    print(f"✓ Validation working: {e}")

# Valid range
current_source.ch1_current(9.5e-6)  # Should work
print(f"✓ Valid value set: {current_source.ch1_current()} A")

## Voltage Source Modules

### `M2m` - 2-Channel Voltage Source  
- **Range**: ±4V with 16-bit resolution
- **Channels**: 2 independent voltage outputs

In [ ]:
# Create voltage sources
voltage_source_m2m = M2m('vs_m2m')
voltage_source_m2m.rack_position('Slot 2')

print(f"M2m voltage source: {voltage_source_m2m.name}")
print(f"Voltage range: {voltage_source_m2m.voltage_range()}")
print(f"Resolution: {voltage_source_m2m.resolution():.2e} V")

# Configure channels
voltage_source_m2m.ch1_enabled(True)
voltage_source_m2m.ch1_voltage(2.5)  # 2.5 V
voltage_source_m2m.ch2_enabled(True)
voltage_source_m2m.ch2_voltage(-1.0)  # -1.0 V

print(f"\nChannel 1: {voltage_source_m2m.ch1_voltage()} V")
print(f"Channel 2: {voltage_source_m2m.ch2_voltage()} V")

### `M2b` - 2-Channel Voltage Source
Similar to M2m with same specifications and parameters

In [ ]:
voltage_source_m2b = M2b('vs_m2b')
voltage_source_m2b.rack_position('Slot 3')
voltage_source_m2b.ch1_voltage(1.5)  # 1.5 V

print(f"M2b voltage source: {voltage_source_m2b.name}")
print(f"Channel 1: {voltage_source_m2b.ch1_voltage()} V")

### `M1b` - 1-Channel Voltage Source
- **Range**: ±4V with 16-bit resolution  
- **Channels**: 1 voltage output

In [ ]:
voltage_source_m1b = M1b('vs_m1b')
voltage_source_m1b.rack_position('Slot 4')
voltage_source_m1b.ch1_voltage(0.8)  # 0.8 V

print(f"M1b voltage source: {voltage_source_m1b.name}")
print(f"Single channel: {voltage_source_m1b.ch1_voltage()} V")
print(f"Has ch2_voltage: {hasattr(voltage_source_m1b, 'ch2_voltage')}")

## Measurement Modules

### `VId` - Multi-Channel Voltage Measurement
- **Range**: ±10V measurement range
- **Channels**: Configurable (default 8 channels)

In [ ]:
# Create voltmeter with custom channel count
voltmeter = VId('vm1', num_channels=4)
voltmeter.rack_position('Slot 5')

print(f"Voltmeter: {voltmeter.name}")
print(f"Number of channels: {voltmeter.num_channels()}")
print(f"Measurement range: {voltmeter.measurement_range()}")

# Simulate some measurements (manually read from device)
measured_voltages = [2.34, -1.56, 0.89, 3.21]  # Example readings
for i, voltage in enumerate(measured_voltages, 1):
    getattr(voltmeter, f'ch{i}_voltage')(voltage)
    print(f"Channel {i}: {getattr(voltmeter, f'ch{i}_voltage')()} V")

## Source-Measure Modules

### `IVd` - Multi-Channel Source-Measure
- **Voltage source**: ±4V with 16-bit resolution
- **Current measurement**: ±100 μA with 16-bit resolution
- **Channels**: Configurable (default 4 channels)

In [ ]:
# Create source-measure unit
smu = IVd('smu1', num_channels=2)
smu.rack_position('Slot 6')

print(f"Source-measure unit: {smu.name}")
print(f"Number of channels: {smu.num_channels()}")
print(f"Voltage range: {smu.voltage_range()}")
print(f"Current range: {smu.current_range()}")
print(f"Voltage resolution: {smu.voltage_resolution():.2e} V")
print(f"Current resolution: {smu.current_resolution():.2e} A")

# Configure source-measure channels
smu.ch1_source_enabled(True)
smu.ch1_voltage(2.0)              # 2.0 V source
smu.ch1_current(50e-6)            # 50 μA measured

smu.ch2_source_enabled(True)
smu.ch2_voltage(-1.5)             # -1.5 V source
smu.ch2_current(-25e-6)           # -25 μA measured

print(f"\nChannel 1 - Source: {smu.ch1_voltage()} V, Measure: {smu.ch1_current()} A")
print(f"Channel 2 - Source: {smu.ch2_voltage()} V, Measure: {smu.ch2_current()} A")

## Practical Usage Example: Measurement Setup

Here's an example of how to configure multiple modules for a typical measurement:

In [ ]:
# Create a complete measurement setup
def setup_ivvi_rack():
    """Set up a complete IVVI rack configuration."""
    
    # Current sources for gate control
    gate_cs = S4c('gate_current_source')
    gate_cs.rack_position('Slot 1')
    gate_cs.notes('Gate voltage control for quantum dots')
    
    # Voltage sources for bias
    bias_vs = M2m('bias_voltage_source')
    bias_vs.rack_position('Slot 2')
    bias_vs.notes('Source-drain bias voltage')
    
    # Voltmeter for monitoring
    monitor_vm = VId('monitor_voltmeter', num_channels=6)
    monitor_vm.rack_position('Slot 3')
    monitor_vm.notes('Voltage monitoring across device')
    
    # Source-measure for current measurement
    current_smu = IVd('current_smu', num_channels=2)
    current_smu.rack_position('Slot 4')
    current_smu.notes('Current measurement through device')
    
    return {
        'gate_cs': gate_cs,
        'bias_vs': bias_vs,
        'monitor_vm': monitor_vm,
        'current_smu': current_smu
    }

# Set up the rack
rack = setup_ivvi_rack()

# Display rack configuration
print("IVVI Rack Configuration:")
print("=" * 40)
for name, module in rack.items():
    idn = module.get_idn()
    print(f"{name}: {idn['vendor']} {idn['model']}")
    print(f"  Position: {module.rack_position()}")
    print(f"  Notes: {module.notes()}")
    print()

### Measurement Loop Example

In [ ]:
# Simulate a gate sweep measurement
def gate_sweep_measurement(rack):
    """Simulate a gate voltage sweep measurement."""
    
    gate_cs = rack['gate_cs']
    current_smu = rack['current_smu']
    
    # Configure initial settings
    gate_cs.ch1_enabled(True)
    current_smu.ch1_source_enabled(False)  # Only measuring
    current_smu.ch1_measure_enabled(True)
    
    # Sweep parameters
    gate_voltages = np.linspace(-2e-6, 2e-6, 11)  # -2 to +2 μA
    
    measurements = []
    
    print("Gate Sweep Measurement:")
    print("Gate Current (μA)\tMeasured Current (nA)")
    print("-" * 45)
    
    for gate_current in gate_voltages:
        # Set gate current (manually on device)
        gate_cs.ch1_current(gate_current)
        
        # Simulate measurement delay
        time.sleep(0.01)
        
        # Simulate measured current (manually read from device)
        # In reality, you would read this from the actual device
        measured_current = 10e-9 * np.sin(gate_current * 1e6) + np.random.normal(0, 0.1e-9)
        current_smu.ch1_current(measured_current)
        
        # Store measurement
        measurement = {
            'gate_current': gate_current,
            'measured_current': measured_current,
            'timestamp': time.time()
        }
        measurements.append(measurement)
        
        print(f"{gate_current*1e6:12.3f}\t\t{measured_current*1e9:12.3f}")
    
    return measurements

# Run the measurement
results = gate_sweep_measurement(rack)
print(f"\nCompleted {len(results)} measurements")

## Module Information and Snapshots

In [ ]:
# Get comprehensive information about a module
def print_module_info(module):
    """Print detailed information about an IVVI module."""
    print(f"Module: {module.name}")
    print(f"Type: {module.module_type()}")
    print(f"Position: {module.rack_position()}")
    print(f"Notes: {module.notes()}")
    
    # Get snapshot for current state
    snapshot = module.snapshot()
    print("\nParameters:")
    for param_name, param_info in snapshot['parameters'].items():
        if 'value' in param_info:
            value = param_info['value']
            unit = param_info.get('unit', '')
            print(f"  {param_name}: {value} {unit}")
    print()

# Show information for each module in the rack
for name, module in rack.items():
    print_module_info(module)
    print("-" * 50)

## Extending the Framework

Adding new IVVI modules is straightforward. Here's an example of how to create a custom module:

In [ ]:
from qcodes.parameters import Parameter
from qcodes.validators import Numbers

class CustomModule(IVVI_Module):
    """Example of a custom IVVI module."""
    
    def __init__(self, name: str, **kwargs):
        super().__init__(name, **kwargs)
        
        # Set module type
        self.module_type('Custom')
        
        # Add custom parameters
        self.add_parameter(
            'custom_parameter',
            initial_value=0.0,
            unit='V',
            vals=Numbers(-10, 10),
            docstring='A custom parameter for demonstration',
            parameter_class=Parameter
        )
        
        self.add_parameter(
            'custom_setting',
            initial_value='default',
            vals=None,
            docstring='A custom setting parameter',
            parameter_class=Parameter
        )

# Create and test the custom module
custom = CustomModule('my_custom_module')
custom.rack_position('Slot 7')
custom.notes('Custom module for special applications')
custom.custom_parameter(5.0)
custom.custom_setting('special_mode')

print_module_info(custom)

## Summary

The QuTech IVVI manual modules provide:

1. **Consistent Interface**: All modules follow the same parameter naming and structure
2. **Parameter Validation**: Automatic range checking with appropriate error messages
3. **Documentation Tools**: Software tracking of manual instrument configurations
4. **QCoDeS Integration**: Seamless integration with QCoDeS measurement scripts
5. **Extensibility**: Easy to add new module types following established patterns

These drivers serve as effective documentation and parameter tracking tools for manual IVVI instruments while maintaining compatibility with existing automated systems using the RS232 driver.

### Available Modules Summary:

- **`IVVI_Module`**: Base class with common functionality
- **`S4c`**: 4-channel current source (±10 μA)
- **`M2m/M2b`**: 2-channel voltage sources (±4V)
- **`M1b`**: 1-channel voltage source (±4V)
- **`VId`**: Multi-channel voltage measurement (±10V)
- **`IVd`**: Multi-channel source-measure (±4V/±100μA)

For electronic control of IVVI modules, use the existing `IVVI` RS232 driver instead.